<a href="https://colab.research.google.com/github/won195/TIL/blob/master/folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.7 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
from pyproj import Proj, transform
import folium

In [7]:
df1 = pd.read_csv('/content/drive/MyDrive/상권분석/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949')

In [8]:


# EPSG 5181 좌표계
proj_epsg5181 = Proj(init='epsg:5181')

# WGS84 좌표계
proj_wgs84 = Proj(init='epsg:4326')

# 새로운 컬럼 생성
df1['lat'] = 0.0
df1['long'] = 0.0

# 좌표 변환
for i, row in df1.iterrows():
    x = row['엑스좌표_값']
    y = row['와이좌표_값']
    lat, long = transform(proj_epsg5181, proj_wgs84, x, y)
    df1.at[i, 'lat'] = lat
    df1.at[i, 'long'] = long

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-8-67d2dade6dfa>:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lat, long = transform(proj_epsg5181, proj_wgs84, x,

In [9]:
df1

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,lat,long
0,202112,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710,NaN,126.908844,37.448687
1,202112,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611,NaN,127.070978,37.634469
2,202112,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680,NaN,126.912978,37.451006
3,202112,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690,NaN,126.901564,37.441482
4,202112,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562,NaN,127.115614,37.509060
...,...,...,...,...,...,...,...,...,...,...,...,...
1666,202112,U,관광특구,1001493,동대문패션타운 관광특구,200974,451975,11140,11140590,NaN,127.011025,37.567312
1667,202112,U,관광특구,1001494,종로?청계 관광특구,199768,452273,11110,11110615,NaN,126.997374,37.569998
1668,202112,U,관광특구,1001495,잠실 관광특구,210191,446339,11710,11710566,NaN,127.115275,37.516476
1669,202112,U,관광특구,1001496,강남 마이스 관광특구,205310,445727,11680,11680580,NaN,127.060059,37.511003


In [20]:
# (1) 지도 생성
m = folium.Map(
    location=[37.56667, 126.97806], # 서울
    zoom_start=13,
    width=1300,
    height=800,

)
for _, row in df1.iterrows():
    folium.Marker(location=[row['long'], row['lat']]).add_to(m)


m

In [21]:
from folium.plugins import MarkerCluster

# (1) 지도 생성
m = folium.Map(
    location=[37.56667, 126.97806], # 서울
    zoom_start=13,
    width=1300,
    height=800,

)


marker_cluster = MarkerCluster().add_to(m)

for _, row in df1.iterrows():
    folium.Marker(location=[row['long'], row['lat']]).add_to(marker_cluster)


m

In [23]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [24]:
from folium.plugins import MarkerCluster

# (1) 지도 생성
m = folium.Map(
    location=[37.56667, 126.97806], # 서울
    zoom_start=13,
    width=1300,
    height=800,

)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)


marker_cluster = MarkerCluster().add_to(m)

for _, row in df1.iterrows():
    folium.Marker(location=[row['long'], row['lat']]).add_to(marker_cluster)


m

In [28]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)



marker_cluster = MarkerCluster().add_to(m)

for _, row in df1.iterrows():
    folium.Marker(location=[row['long'], row['lat']]).add_to(marker_cluster)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m
